In [1]:
import sys
import os
from warnings import warn

notebooks_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebooks_dir, ".."))
sys.path.append(project_root)

from scripts.lang_embeddings import language_tokenizer

/nix/store/zyr273098nb8bh539d3fmz8vil3c3inz-python3.12-tqdm-4.67.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pickle
import pandas as pd
import numpy as np

df = pd.read_csv("../data/translators.csv")

In [3]:
collapsed_df = (
    df.groupby("TRANSLATOR")
    .agg(
        SOURCE_LANG=("SOURCE_LANG", lambda x: language_tokenizer(x.mode()[0])),
        TARGET_LANG=("TARGET_LANG", lambda x: language_tokenizer(x.mode()[0])),
        HOURLY_RATE=("HOURLY_RATE", "mean"),
    )
    .reset_index()
)

# Display the resulting DataFrame
collapsed_df

,TRANSLATOR,SOURCE_LANG,TARGET_LANG,HOURLY_RATE
0,Aaron,English,Basque,20.700000
1,Abdon,English,Spanish_LA,18.500000
2,Abdon Isaias,English,Spanish_Iberian,16.375000
3,Abdon Luis,English,Spanish_Iberian,16.000000
4,Abel Irene,English,Spanish_Iberian,17.000000
...,...,...,...,...
978,Zacarias Casio,English,Spanish_Argentina,18.181818
979,Zacarias Marcelino,English,Spanish_Global,20.600000
980,Zachary,Catalan,English,22.000000
981,Zlatan,English,Swedish,40.000000


In [4]:
def get_embedding(lang):
    idx = lang_to_index.get(lang)
    if idx is not None:
        return embeddings[idx]
    else:
        return np.zeros(embeddings.shape[1])

In [5]:
def add_most_frequent_category(df: pd.DataFrame, target_df: pd.DataFrame, group_col: str, category_cols: list) -> pd.DataFrame:
    merged_df = target_df.copy()

    for col in category_cols:
        if col not in df.columns:
            warn(f"Warning: Column '{col}' not found in the input DataFrame. Skipping.")
            continue

        count_series = df.groupby([group_col, col]).size().reset_index(name="count")
        idx_max_count = count_series.groupby(group_col)["count"].idxmax()

        most_frequent_categories = count_series.loc[idx_max_count]
        most_frequent_categories = most_frequent_categories[[group_col, col]]

        merged_df = pd.merge(
            merged_df,
            most_frequent_categories,
            on=group_col,
            how="left" # Use left merge to keep all rows from target_df
        )
        print(f"Merged most frequent '{col}' into target DataFrame.")


    return merged_df


def add_mean_numerical_value(df: pd.DataFrame, target_df: pd.DataFrame, group_col: str, numerical_cols: list) -> pd.DataFrame:
    merged_df = target_df.copy()

    for col in numerical_cols:
        if col not in df.columns:
            warn(f"Warning: Column '{col}' not found in the input DataFrame. Skipping.")
            continue

        if not pd.api.types.is_numeric_dtype(df[col]):
             warn(f"Warning: Column '{col}' is not of numeric dtype. Skipping mean calculation.")
             continue

        mean_values = df.groupby(group_col)[col].mean().reset_index()

        new_col_name = f'{col}_mean'
        mean_values.rename(columns={col: new_col_name}, inplace=True)

        merged_df = pd.merge(
            merged_df,
            mean_values,
            on=group_col,
            how="left",
        )
        print(f"Merged mean of '{col}' into target DataFrame as '{new_col_name}'.")

    return merged_df

In [6]:
df_task = pd.read_csv("../data/data_enhanced.csv")
df_task

,Unnamed: 0,PROJECT_ID,PM,TASK_ID,START,END,TASK_TYPE,SOURCE_LANG,TARGET_LANG,TRANSLATOR,...,QUALITY_EVALUATION,MANUFACTURER,MANUFACTURER_SECTOR,MANUFACTURER_INDUSTRY_GROUP,MANUFACTURER_INDUSTRY,MANUFACTURER_SUBINDUSTRY,_work_ready,_time_taken,_time_reception,_time_to_close
0,0,213494,KMT,10048285,26/11/2014 10:36:00,26/11/2014 15:30:00,Engineering,English,Portuguese (Brazil),Estela,...,7,Coastal Cottage,Consumer Discretionary,Consumer Services,"Hotels, Restaurants & Leisure","Hotels, Resorts & Cruise Lines",0.000556,1.261111,0.027500,0.000556
1,1,214198,KMT,10048285,09/09/2015 17:29:00,10/09/2015 11:00:00,Engineering,English,Spanish (Iberian),Jeronimo,...,7,HealthyLife,Health Care,Health Care Providers,Health Care Facilities,Long-Term Care Facilities,0.014444,23.109167,0.845556,0.000556
2,2,213094,PMT,10048285,01/05/2014 19:27:00,02/05/2014 19:00:00,Engineering,Catalan,Catalan,Octavi,...,6,AeroSysTech,Information Technology,Software & Services,Application Software,Systems Software,0.000556,0.000556,0.000833,0.000556
3,3,211967,KMT,10048285,17/07/2013 17:40:00,31/08/2013 18:00:00,Management,English,Spanish (Iberian),Ramiro Josafat,...,6,MetaPro,Industrials,Electrical Equipment,Electrical Components & Equipment,Electrical Components & Equipment,936.324444,0.000833,24.552778,0.001111
4,4,212331,PMT,10048285,01/11/2013 13:13:00,31/10/2013 19:00:00,Miscellaneous,Catalan,Catalan,Victor,...,8,SunTech,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals",0.000556,0.000556,0.000556,0.000833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31012,31012,209001,BMT,10146479,03/05/2011 19:00:00,12/05/2011 17:00:00,ProofReading,English,Spanish (Iberian),Salma,...,8,FrontierTech,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals",2.887500,171.250556,15.116389,0.000833
31013,31013,209034,PMT,10146480,02/05/2011 13:27:00,03/05/2011 11:00:00,Translation,English,Spanish (Iberian),Fiamma Baldomero,...,7,TechStream,Information Technology,IT Services,IT Consulting & Other Services,IT Consulting & Other Services,0.628333,21.006944,1.653889,0.000000
31014,31014,209034,PMT,10146481,03/05/2011 11:00:00,03/05/2011 14:00:00,ProofReading,English,Spanish (Iberian),Gisela Ildefonso,...,6,TechStream,Information Technology,IT Services,IT Consulting & Other Services,IT Consulting & Other Services,0.441111,0.936111,0.276667,0.000000
31015,31015,208898,BMT,10146482,02/05/2011 13:45:00,02/05/2011 18:00:00,Translation,English,Spanish (Iberian),Ramiro Josafat,...,8,FrontierTech,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals",1.023056,0.000833,0.023611,0.000833


In [7]:
category_columns = [
    "MANUFACTURER_INDUSTRY",
    "TASK_TYPE",
    "PM",
    ]
translators_df = add_most_frequent_category(
    df=df_task,
    target_df=collapsed_df,
    group_col="TRANSLATOR",
    category_cols=category_columns
)

numerical_columns = [
    "FORECAST",
    "HOURLY_RATE",
    "QUALITY_EVALUATION",
]

translators_df = add_mean_numerical_value(
    df=df_task,
    target_df=translators_df,
    group_col="TRANSLATOR",
    numerical_cols=numerical_columns,
)

Merged most frequent 'MANUFACTURER_INDUSTRY' into target DataFrame.
Merged most frequent 'TASK_TYPE' into target DataFrame.
Merged most frequent 'PM' into target DataFrame.
Merged mean of 'FORECAST' into target DataFrame as 'FORECAST_mean'.
Merged mean of 'HOURLY_RATE' into target DataFrame as 'HOURLY_RATE_mean'.
Merged mean of 'QUALITY_EVALUATION' into target DataFrame as 'QUALITY_EVALUATION_mean'.


In [8]:
# drop the rows that have missing values on MANUFACTURER_INDUSTRY and TASK_TYPE
translators_df = translators_df[
    ~translators_df["MANUFACTURER_INDUSTRY"].isnull()
]

renamed = {
    'HOURLY_RATE_x': 'HOURLY_RATE',
    'HOURLY_RATE_y': 'HOURLY_RATE_AVG_TASK',
}

translators_df.rename(columns=renamed, inplace=True)

translators_df

,TRANSLATOR,SOURCE_LANG,TARGET_LANG,HOURLY_RATE,MANUFACTURER_INDUSTRY,TASK_TYPE,PM,FORECAST_mean,HOURLY_RATE_mean,QUALITY_EVALUATION_mean
0,Aaron,English,Basque,20.700000,Internet Services & Infrastructure,ProofReading,RMT,0.500000,19.500000,4.000000
2,Abdon Isaias,English,Spanish_Iberian,16.375000,"Technology Hardware, Storage & Peripherals",Translation,BMT,4.094280,16.964337,7.069900
5,Abelardo,English,Catalan,20.555556,Communications Equipment,Translation,RMT,7.033418,22.075949,7.113924
11,Acacio Poncio,English,Spanish_Iberian,18.727273,Application Software,Translation,BMT,4.994358,18.576375,6.871690
26,Agueda,English,Catalan,21.476190,Internet Services & Infrastructure,Translation,RMT,2.951667,22.482301,6.967552
...,...,...,...,...,...,...,...,...,...,...
962,Vidal Liduvina,Catalan,Spanish_Global,20.000000,Health Care Facilities,ProofReading,PMT,4.765556,20.000000,7.222222
968,Wolfgang,English,German,29.000000,Semiconductors & Semiconductor Equipment,Translation,PMT,0.960000,29.000000,7.000000
971,Xoana,English,Spanish_Iberian,15.176471,Internet Services & Infrastructure,ProofReading,RMT,8.513830,15.244681,7.053191
975,Yoselin Teofila,English,Portuguese_Brazil,21.000000,Trading Companies & Distributors,Translation,PMT,2.315000,25.750000,7.875000


By looking at the DF, we can verify that there are no missing values on the columns.

### Embed Language

From our pre-made embeddings

In [9]:
# the languages are already in our desided format

from scripts.lang_embeddings import EmbeddingLookup, load_embedding_data

embedding_data = load_embedding_data("../scripts/language_embeddings.pkl")

lookup = EmbeddingLookup(loaded_data=embedding_data)

Successfully loaded embedding data from '../scripts/language_embeddings.pkl'
EmbeddingLookup initialized with 79 languages.
  Original embedding dimension: 384
  Latent embedding dimension: 10


In [10]:
source_lang_embed = translators_df['SOURCE_LANG'].apply(lookup.get_vector)
target_lang_embed = translators_df['TARGET_LANG'].apply(lookup.get_vector)

translators_df['SOURCE_LANG_EMBED'] = source_lang_embed
translators_df['TARGET_LANG_EMBED'] = target_lang_embed
translators_df

,TRANSLATOR,SOURCE_LANG,TARGET_LANG,HOURLY_RATE,MANUFACTURER_INDUSTRY,TASK_TYPE,PM,FORECAST_mean,HOURLY_RATE_mean,QUALITY_EVALUATION_mean,SOURCE_LANG_EMBED,TARGET_LANG_EMBED
0,Aaron,English,Basque,20.700000,Internet Services & Infrastructure,ProofReading,RMT,0.500000,19.500000,4.000000,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.033526402, -0.052197512, 0.015763432, -0.01..."
2,Abdon Isaias,English,Spanish_Iberian,16.375000,"Technology Hardware, Storage & Peripherals",Translation,BMT,4.094280,16.964337,7.069900,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899..."
5,Abelardo,English,Catalan,20.555556,Communications Equipment,Translation,RMT,7.033418,22.075949,7.113924,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.049495008, -0.07142061, -0.005828036, -0.01..."
11,Acacio Poncio,English,Spanish_Iberian,18.727273,Application Software,Translation,BMT,4.994358,18.576375,6.871690,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899..."
26,Agueda,English,Catalan,21.476190,Internet Services & Infrastructure,Translation,RMT,2.951667,22.482301,6.967552,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.049495008, -0.07142061, -0.005828036, -0.01..."
...,...,...,...,...,...,...,...,...,...,...,...,...
962,Vidal Liduvina,Catalan,Spanish_Global,20.000000,Health Care Facilities,ProofReading,PMT,4.765556,20.000000,7.222222,"[0.049495008, -0.07142061, -0.005828036, -0.01...","[0.05821279, -0.058210686, 0.13881628, -0.0406..."
968,Wolfgang,English,German,29.000000,Semiconductors & Semiconductor Equipment,Translation,PMT,0.960000,29.000000,7.000000,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[-0.013138592, -0.04253142, 0.02580829, -0.029..."
971,Xoana,English,Spanish_Iberian,15.176471,Internet Services & Infrastructure,ProofReading,RMT,8.513830,15.244681,7.053191,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899..."
975,Yoselin Teofila,English,Portuguese_Brazil,21.000000,Trading Companies & Distributors,Translation,PMT,2.315000,25.750000,7.875000,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.061353415, -0.025645662, 0.11521641, -0.058..."


### Embed Industry

From our pre-made embeddings

In [11]:
from scripts.industry_embeddings import EmbeddingLookup, load_embedding_data, industry_tokenizer
embedding_data = load_embedding_data("../scripts/industry_embeddings.pkl")

lookup = EmbeddingLookup(loaded_data=embedding_data)

Successfully loaded embedding data from '../scripts/industry_embeddings.pkl'
EmbeddingLookup initialized with 85 languages.
  Original embedding dimension: 384
  Latent embedding dimension: 10


In [12]:
industry_embed = translators_df['MANUFACTURER_INDUSTRY'].apply(industry_tokenizer).apply(lookup.get_vector)
translators_df['INDUSTRY_EMBED'] = industry_embed
translators_df

,TRANSLATOR,SOURCE_LANG,TARGET_LANG,HOURLY_RATE,MANUFACTURER_INDUSTRY,TASK_TYPE,PM,FORECAST_mean,HOURLY_RATE_mean,QUALITY_EVALUATION_mean,SOURCE_LANG_EMBED,TARGET_LANG_EMBED,INDUSTRY_EMBED
0,Aaron,English,Basque,20.700000,Internet Services & Infrastructure,ProofReading,RMT,0.500000,19.500000,4.000000,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.033526402, -0.052197512, 0.015763432, -0.01...","[-0.065250605, 0.029024575, 0.032841533, -0.06..."
2,Abdon Isaias,English,Spanish_Iberian,16.375000,"Technology Hardware, Storage & Peripherals",Translation,BMT,4.094280,16.964337,7.069900,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...","[0.005651269, 0.056426913, -0.016729122, -0.06..."
5,Abelardo,English,Catalan,20.555556,Communications Equipment,Translation,RMT,7.033418,22.075949,7.113924,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.049495008, -0.07142061, -0.005828036, -0.01...","[-0.009626694, 0.019526087, -0.027109567, -0.0..."
11,Acacio Poncio,English,Spanish_Iberian,18.727273,Application Software,Translation,BMT,4.994358,18.576375,6.871690,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...","[-0.051018845, 0.013300572, 0.061036862, -0.02..."
26,Agueda,English,Catalan,21.476190,Internet Services & Infrastructure,Translation,RMT,2.951667,22.482301,6.967552,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.049495008, -0.07142061, -0.005828036, -0.01...","[-0.065250605, 0.029024575, 0.032841533, -0.06..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,Vidal Liduvina,Catalan,Spanish_Global,20.000000,Health Care Facilities,ProofReading,PMT,4.765556,20.000000,7.222222,"[0.049495008, -0.07142061, -0.005828036, -0.01...","[0.05821279, -0.058210686, 0.13881628, -0.0406...","[-0.060847882, -0.16297893, 0.06373858, -0.032..."
968,Wolfgang,English,German,29.000000,Semiconductors & Semiconductor Equipment,Translation,PMT,0.960000,29.000000,7.000000,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[-0.013138592, -0.04253142, 0.02580829, -0.029...","[0.038477384, 0.028468098, -0.03593983, -0.082..."
971,Xoana,English,Spanish_Iberian,15.176471,Internet Services & Infrastructure,ProofReading,RMT,8.513830,15.244681,7.053191,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...","[-0.065250605, 0.029024575, 0.032841533, -0.06..."
975,Yoselin Teofila,English,Portuguese_Brazil,21.000000,Trading Companies & Distributors,Translation,PMT,2.315000,25.750000,7.875000,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.061353415, -0.025645662, 0.11521641, -0.058...","[-0.019893985, 0.08402714, -0.0744723, -0.0272..."


### Save the dataframe

In [13]:
translators_df.to_pickle(
    "../data/translators_enhanced.pkl",
)